In [2]:
from google.colab import drive
drive.mount('/gdrive/')
%cd /gdrive/My Drive/Colab Notebooks/gesture/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive/
/gdrive/My Drive/Colab Notebooks/gesture


In [0]:
import os
import csv
import numpy as np
import random
import pandas as pd
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score,accuracy_score
import matplotlib.pyplot as plt

# Load data

In [5]:
csv_lines = [] #preallocate list for extracted csv lines

path_to_csv = "dataset.csv"

raw_data = pd.read_csv(path_to_csv, sep=';')

# encode as nominal
#raw_data.user.unique(); raw_data.user = raw_data.user.map({'debora':0, 'katia':1, 'wallace':2, 'jose_carlos':3}); raw_data.user.unique()
raw_data.gender.unique(); raw_data.gender = raw_data.gender.map({'Woman':1, 'Man':0}); raw_data.gender.unique()

#raw_data.class.unique(); raw_data.class = raw_data.class.map({'sitting':0, 'sittingdown':1, 'standing':2, 'standingup':3, 'walking:4'}); raw_data.class.unique()


raw_data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user,gender,age,how_tall_in_meters,weight,body_mass_index,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,y4,z4,class
0,debora,1,46,"1,62",75,"28,6",-3,92,-63,-23,18,-19,5,104,-92,-150,-103,-147,sitting
1,debora,1,46,"1,62",75,"28,6",-3,94,-64,-21,18,-18,-14,104,-90,-149,-104,-145,sitting
2,debora,1,46,"1,62",75,"28,6",-1,97,-61,-12,20,-15,-13,104,-90,-151,-104,-144,sitting
3,debora,1,46,"1,62",75,"28,6",-2,96,-57,-15,21,-16,-13,104,-89,-153,-103,-142,sitting
4,debora,1,46,"1,62",75,"28,6",-1,96,-61,-13,20,-15,-13,104,-89,-153,-104,-143,sitting


# Replace , with . in data

In [6]:
raw_data['how_tall_in_meters'] = raw_data['how_tall_in_meters'].str.replace(',', '.')
raw_data['body_mass_index'] = raw_data['body_mass_index'].str.replace(',', '.')

raw_data.head()

,user,gender,age,how_tall_in_meters,weight,body_mass_index,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,y4,z4,class
0,debora,1,46,1.62,75,28.6,-3,92,-63,-23,18,-19,5,104,-92,-150,-103,-147,sitting
1,debora,1,46,1.62,75,28.6,-3,94,-64,-21,18,-18,-14,104,-90,-149,-104,-145,sitting
2,debora,1,46,1.62,75,28.6,-1,97,-61,-12,20,-15,-13,104,-90,-151,-104,-144,sitting
3,debora,1,46,1.62,75,28.6,-2,96,-57,-15,21,-16,-13,104,-89,-153,-103,-142,sitting
4,debora,1,46,1.62,75,28.6,-1,96,-61,-13,20,-15,-13,104,-89,-153,-104,-143,sitting


# check if data have NULL value or not.

In [7]:
raw_data[pd.isnull(raw_data).any(axis=1)]

raw_data.isnull().values.any(); raw_data.isnull().sum().sum()

raw_data.shape

(165633, 19)

In [8]:
raw_data.drop(raw_data[raw_data.z4 == "-14420-11-2011 04:50:23,713"].index.values, inplace=True) # row 122076 -> (165633, 19)
raw_data.z4 = pd.to_numeric(raw_data.z4, errors='raise'); raw_data.dtypes # z4 object -> int64
raw_data.columns

raw_data[pd.isnull(raw_data).any(axis=1)] # show missing column

,user,gender,age,how_tall_in_meters,weight,body_mass_index,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,y4,z4,class
163840,jose_carlos,0,75,1.67,67,NaN,2,96,-126,41,51,-65,3,123,-113,-200,-87,-161,walking
163841,jose_carlos,0,75,1.67,67,NaN,-23,83,-137,-28,105,-131,0,160,-117,-195,-58,-149,walking
163842,jose_carlos,0,75,1.67,67,NaN,-10,87,-145,-25,86,-100,77,12,-84,-186,-87,-139,walking
163843,jose_carlos,0,75,1.67,67,NaN,-17,88,-144,-18,86,-96,0,95,-114,-189,-85,-142,walking
163844,jose_carlos,0,75,1.67,67,NaN,-11,103,-136,-14,92,-93,38,121,-97,-189,-92,-144,walking
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165628,jose_carlos,0,75,1.67,67,NaN,2,93,-148,-8,88,-100,15,120,-100,-189,-77,-160,walking
165629,jose_carlos,0,75,1.67,67,NaN,-1,94,-147,-17,87,-99,17,121,-99,-190,-78,-158,walking
165630,jose_carlos,0,75,1.67,67,NaN,-10,93,-143,-19,86,-104,0,114,-101,-185,-80,-153,walking
165631,jose_carlos,0,75,1.67,67,NaN,-15,86,-152,-19,88,-117,-44,155,-25,-185,-84,-156,walking


In [9]:
# Missing data count

raw_data = raw_data[raw_data['body_mass_index'].notnull()]

raw_data[pd.isnull(raw_data).any(axis=1)]

raw_data.isnull().values.any(); raw_data.isnull().sum().sum()

0

# Y label

In [0]:
y = raw_data['class'].copy()
type(y); y.describe(); y.head(3); y.shape; y.unique() # (165632, )

# y = pd.get_dummies(y, columns=['class'])

y = y.map({'sitting':0, 'sittingdown':1, 'standing':2, 'standingup':3, 'walking':4})

In [11]:
raw_data.drop(labels=['user', 'class'], axis=1, inplace=True); raw_data.shape # (165632, 17)
raw_data.head()

,gender,age,how_tall_in_meters,weight,body_mass_index,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,y4,z4
0,1,46,1.62,75,28.6,-3,92,-63,-23,18,-19,5,104,-92,-150,-103,-147
1,1,46,1.62,75,28.6,-3,94,-64,-21,18,-18,-14,104,-90,-149,-104,-145
2,1,46,1.62,75,28.6,-1,97,-61,-12,20,-15,-13,104,-90,-151,-104,-144
3,1,46,1.62,75,28.6,-2,96,-57,-15,21,-16,-13,104,-89,-153,-103,-142
4,1,46,1.62,75,28.6,-1,96,-61,-13,20,-15,-13,104,-89,-153,-104,-143


# Split data to train/ test by Hold-out

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(raw_data, y, test_size=0.30, random_state=7)
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

((114687, 17), (49152, 17), (114687,), (49152,))

# KNN approach

In [20]:
model = KNeighborsClassifier(n_neighbors=16)

model.fit(X_train, y_train)
y_test_predict = model.predict(X_test)
fscore = f1_score(y_test_predict.astype('uint8'), y_test.astype('uint8'), average='micro')
accu = accuracy_score(y_test_predict.astype('uint8'), y_test.astype('uint8'))
print('Fscore=', fscore)
print('Accuracy=', accu)

Fscore= 0.9900716145833334
Accuracy= 0.9900716145833334


# Hyperparameter finding

#### 1 - Cosine similarity KNN

In [14]:
max_fscore = 0
best_neighbor = 0
print("Finding best \"neighbor\" hyperparam..")
for neighbor in range(1,30):
    model = KNeighborsClassifier(n_neighbors=neighbor, metric='cosine')
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    fscore = f1_score(y_test_pred.astype('uint8'), y_test.astype('uint8'), average='micro')
    if max_fscore < fscore:
        max_fscore = fscore
        best_neighbor = neighbor
        accu = accuracy_score(y_test_pred.astype('uint8'), y_test.astype('uint8'))

        
print('Best neighbor=',best_neighbor)
print('Best fscore=',max_fscore)
print('Accuracy=',accu)

Finding best "neighbor" hyperparam..
Best neighbor= 1
Best fscore= 0.9952189127604166
Accuracy= 0.9952189127604166


#### 2- Simple approach finding k from 1 - 30

In [15]:
max_fscore = 0
best_neighbor = 0
print("Finding best \"neighbor\" hyperparam..")
for neighbor in range(1,30):
    model = KNeighborsClassifier(n_neighbors=neighbor)
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    fscore = f1_score(y_test_pred.astype('uint8'), y_test.astype('uint8'), average='micro')
    if max_fscore < fscore:
        max_fscore = fscore
        best_neighbor = neighbor
        accu = accuracy_score(y_test_pred.astype('uint8'), y_test.astype('uint8'))

        
print('Best neighbor=',best_neighbor)
print('Best fscore=',max_fscore)
print('Accuracy=',accu)

Finding best "neighbor" hyperparam..
Best neighbor= 1
Best fscore= 0.9953409830729166
Accuracy= 0.9953409830729166


#### 3 - Euclid Metric

In [16]:
max_fscore = 0
best_neighbor = 0
print("Finding best \"neighbor\" hyperparam..")
for neighbor in range(1,30):
    model = KNeighborsClassifier(n_neighbors=neighbor, metric='euclidean')
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    fscore = f1_score(y_test_pred.astype('uint8'), y_test.astype('uint8'), average='micro')
    if max_fscore < fscore:
        max_fscore = fscore
        best_neighbor = neighbor
        accu = accuracy_score(y_test_pred.astype('uint8'), y_test.astype('uint8'))

        
print('Best neighbor=',best_neighbor)
print('Best fscore=',max_fscore)
print('Accuracy=',accu)


Finding best "neighbor" hyperparam..
Best neighbor= 1
Best fscore= 0.9953409830729166
Accuracy= 0.9953409830729166


#### 4 - Manhattan Metric

In [17]:
max_fscore = 0
best_neighbor = 0
print("Finding best \"neighbor\" hyperparam..")
for neighbor in range(1,30):
    model = KNeighborsClassifier(n_neighbors=neighbor, metric='euclidean')
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    fscore = f1_score(y_test_pred.astype('uint8'), y_test.astype('uint8'), average='micro')
    if max_fscore < fscore:
        max_fscore = fscore
        best_neighbor = neighbor
        accu = accuracy_score(y_test_pred.astype('uint8'), y_test.astype('uint8'))

        
print('Best neighbor=',best_neighbor)
print('Best fscore=',max_fscore)
print('Accuracy=',accu)


Finding best "neighbor" hyperparam..
Best neighbor= 1
Best fscore= 0.9953409830729166
Accuracy= 0.9953409830729166
